# Understanding Ontologies with Cognee

This notebook demonstrates how to work with ontologies in scientific research using the Cognee framework. We'll explore how ontologies can enhance our understanding and querying of scientific papers.

## What is an Ontology?

An ontology is a formal representation of knowledge that defines:
- Concepts within a domain
- Relationships between concepts
- Properties and attributes
- Rules and constraints

Key terms:
- **Classes**: Categories or types (e.g., Disease, Symptom)
- **Instances**: Specific examples of classes (e.g., Type 2 Diabetes)
- **Properties**: Relationships between classes/instances (e.g., hasSymptom)
- **Axioms**: Logical statements defining relationships

## Setup

First, let's install the required packages and set up our environment:

In [ ]:
# Install required package
# !pip install cognee

In [1]:
import os

# Set up OpenAI API key (required for Cognee's LLM functionality)
if "LLM_API_KEY" not in os.environ:
    os.environ["LLM_API_KEY"] = "your-api-key-here"  # Replace with your API key

In [2]:
# Import required libraries
import cognee
print(cognee.__version__)
from cognee.shared.logging_utils import get_logger

cognee.config.set_llm_model("gpt-4o-mini")
cognee.config.set_llm_provider("openai")
from cognee.api.v1.search import SearchType

logger = get_logger()


2025-10-07T20:40:15.192965 [info     ] Deleted old log file: /Users/daulet/Desktop/dev/cognee-claude/logs/2025-10-07_21-25-04.log [cognee.shared.logging_utils]

2025-10-07T20:40:15.894155 [info     ] Logging initialized            [cognee.shared.logging_utils] cognee_version=0.3.5-local database_path=/Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases graph_database_name= os_info='Darwin 24.5.0 (Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:43 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T8132)' python_version=3.10.11 relational_config=cognee_db structlog_version=25.4.0 vector_config=lancedb

2025-10-07T20:40:15.894641 [info     ] Database storage: /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases [cognee.shared.logging_utils]
/Users/daulet/Desktop/dev/cognee-claude/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_

0.3.5-local


## Creating the Pipeline

Let's create a pipeline that will:
1. Clean existing data
2. Process scientific papers
3. Apply ontological knowledge

In [3]:
async def run_pipeline(config=None):
    # Clean existing data
    await cognee.prune.prune_data()
    await cognee.prune.prune_system(metadata=True)
    
    # Set up path to scientific papers
    scientific_papers_dir = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath("."))), 
        "cognee",
        "examples",
        "data", 
        "scientific_papers/"
    )
    
    # Add papers to the system
    await cognee.add(scientific_papers_dir)
    
    # Cognify with optional ontology
    return await cognee.cognify(config=config)

async def query_pipeline(questions):
    answers = []
    for question in questions:
        search_results = await cognee.search(
            query_type=SearchType.GRAPH_COMPLETION,
            query_text=question,
        )
        answers.append(search_results)
    return answers

## Running the Demo

Let's test our system with some medical questions, comparing results with and without ontological knowledge:

In [4]:
from cognee.modules.ontology.rdf_xml.RDFLibOntologyResolver import RDFLibOntologyResolver
from cognee.modules.ontology.ontology_config import Config
# Test questions
questions = [
    "What are common risk factors for Type 2 Diabetes?",
    "What preventive measures reduce the risk of Hypertension?",
    "What symptoms indicate possible Cardiovascular Disease?",
    "What diseases are associated with Obesity?"
]

# Path to medical ontology
ontology_path = "../examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl"  # Update with your ontology path

config: Config = {
        "ontology_config": {
            "ontology_resolver": RDFLibOntologyResolver(ontology_file=ontology_path)
        }
    }

# Run with ontology
print("\n--- Results WITH ontology ---\n")
await run_pipeline(config=config)
answers_with = await query_pipeline(questions)
for q, a in zip(questions, answers_with):
    print(f"Q: {q}\nA: {a}\n")


2025-10-07T20:40:24.236015 [info     ] Ontology loaded successfully from file: ../examples/python/ontology_input_example/enriched_medical_ontology_with_classes.owl [OntologyAdapter]

2025-10-07T20:40:24.236931 [info     ] Lookup built: 4 classes, 50 individuals [OntologyAdapter]

2025-10-07T20:40:24.348030 [info     ] JSON extension already loaded or unavailable: Binder exception: Extension: JSON is already loaded. You can check loaded extensions by `CALL SHOW_LOADED_EXTENSIONS() RETURN *`. [cognee.shared.logging_utils]

2025-10-07T20:40:24.387417 [info     ] Deleted Kuzu database files at /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases/cognee_graph_kuzu [cognee.shared.logging_utils]



--- Results WITH ontology ---




2025-10-07T20:40:26.930731 [info     ] Database deleted successfully. [cognee.shared.logging_utils]

Storage manager absolute path: /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_cache

Deleting cache...             

✓ Cache deleted successfully! 


User ddfe2676-fa68-430d-981e-1d335a6fdb1b has registered.



EmbeddingRateLimiter initialized: enabled=False, requests_limit=60, interval_seconds=60

2025-10-07T20:40:28.202666 [info     ] Pipeline run started: `00128a51-0bd2-5512-9865-851caf7251ba` [run_tasks_with_telemetry()]

2025-10-07T20:40:28.203107 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-10-07T20:40:28.203471 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-10-07T20:40:28.204254 [info     ] Pipeline run started: `00128a51-0bd2-5512-9865-851caf7251ba` [run_tasks_with_telemetry()]

2025-10-07T20:40:28.204491 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-10-07T20:40:28.204812 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-10-07T20:40:28.219424 [info     ] Registered loader: pypdf_loader [cognee.infrastructure.loaders.LoaderEngine]

2025-10-07T20:40:28.219844 [info     ] Registered loader: text_loader [cognee.infrastructure.loaders.LoaderEngine]

2025-10-07T


Reading PDF for file: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/nutrients-13-01241.pdf ...

2025-10-07T20:40:35.976082 [warning  ] Failed to process PDF with AdvancedPdfLoader: Unable to get page count. Is poppler installed and in PATH? [cognee.infrastructure.loaders.external.advanced_pdf_loader]

2025-10-07T20:40:35.977344 [info     ] Falling back to PyPDF loader for /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/nutrients-13-01241.pdf [cognee.infrastructure.loaders.external.advanced_pdf_loader]

2025-10-07T20:40:35.979542 [info     ] Reading PDF: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/nutrients-13-01241.pdf [cognee.infrastructure.loaders.external.pypdf_loader]

2025-10-07T20:40:36.323535 [info     ] Processing PDF: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/TOJ-22-0073_152Mendoza.pdf [cognee.infrastructure.loaders.external.advanced_pdf_loader]

Reading PDF for file: /Users/daulet/Desktop/dev/cog


2025-10-07T20:40:36.849367 [info     ] Coroutine task started: `extract_graph_from_data` [run_tasks_base]

2025-10-07T20:40:36.863627 [info     ] Coroutine task started: `extract_graph_from_data` [run_tasks_base]

2025-10-07T20:40:55.726911 [info     ] Loaded JSON extension          [cognee.shared.logging_utils]

2025-10-07T20:40:55.757864 [info     ] No close match found for 'person' in category 'classes' [OntologyAdapter]

2025-10-07T20:40:55.758561 [info     ] No close match found for 'michael f. mendoza' in category 'individuals' [OntologyAdapter]

2025-10-07T20:40:55.759088 [info     ] No close match found for 'ralf martz sulague' in category 'individuals' [OntologyAdapter]

2025-10-07T20:40:55.759456 [info     ] No close match found for 'therese posas-mendoza' in category 'individuals' [OntologyAdapter]

2025-10-07T20:40:55.759801 [info     ] No close match found for 'carl j. lavie' in category 'individuals' [OntologyAdapter]

2025-10-07T20:40:55.760065 [info     ] No close matc

Q: What are common risk factors for Type 2 Diabetes?
A: ['Common risk factors for Type 2 Diabetes include:\n1. Obesity\n2. Hypertension (high blood pressure)\n3. High cholesterol\n4. Smoking\n5. Cardiovascular disease\n6. Heart failure']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preventive measures to reduce the risk of hypertension include:\n1. Low sodium diet\n2. Moderate coffee consumption\n3. Regular exercise\n4. Maintaining a healthy lifestyle']

Q: What symptoms indicate possible Cardiovascular Disease?
A: ['Symptoms indicating possible cardiovascular disease include:\n1. Chest pain\n2. Shortness of breath\n3. Fatigue']

Q: What diseases are associated with Obesity?
A: ['Diseases associated with obesity include cardiovascular disease, diabetes, hypertension, high cholesterol, and high blood pressure.']




2025-10-07T20:41:48.134301 [info     ] Loaded JSON extension          [cognee.shared.logging_utils]


In [5]:
# Run without ontology
print("\n--- Results WITHOUT ontology ---\n")
await run_pipeline()
answers_without = await query_pipeline(questions)
for q, a in zip(questions, answers_without):
    print(f"Q: {q}\nA: {a}\n")


2025-10-07T20:41:23.957345 [info     ] Deleted Kuzu database files at /Users/daulet/Desktop/dev/cognee-claude/cognee/.cognee_system/databases/cognee_graph_kuzu [cognee.shared.logging_utils]



--- Results WITHOUT ontology ---




2025-10-07T20:41:25.980678 [info     ] Database deleted successfully. [cognee.shared.logging_utils]

Deleting cache...             

✓ Cache deleted successfully! 

2025-10-07T20:41:26.100988 [info     ] Pipeline run started: `d7e0340c-b6c6-568f-856c-b9f4347628d4` [run_tasks_with_telemetry()]

2025-10-07T20:41:26.101412 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-10-07T20:41:26.101746 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-10-07T20:41:26.102449 [info     ] Pipeline run started: `d7e0340c-b6c6-568f-856c-b9f4347628d4` [run_tasks_with_telemetry()]

2025-10-07T20:41:26.102713 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-10-07T20:41:26.103039 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-10-07T20:41:26.115879 [info     ] Processing PDF: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/TOJ-22-0073_152Mendoza.pdf [cognee.infrastructur

User 215612a9-f107-44d8-9263-d680872182c9 has registered.



Reading PDF for file: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/TOJ-22-0073_152Mendoza.pdf ...

2025-10-07T20:41:26.605043 [warning  ] Failed to process PDF with AdvancedPdfLoader: Unable to get page count. Is poppler installed and in PATH? [cognee.infrastructure.loaders.external.advanced_pdf_loader]

2025-10-07T20:41:26.605986 [info     ] Falling back to PyPDF loader for /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/TOJ-22-0073_152Mendoza.pdf [cognee.infrastructure.loaders.external.advanced_pdf_loader]

2025-10-07T20:41:26.606982 [info     ] Reading PDF: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/TOJ-22-0073_152Mendoza.pdf [cognee.infrastructure.loaders.external.pypdf_loader]

2025-10-07T20:41:26.714728 [info     ] Processing PDF: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/nutrients-13-01241.pdf [cognee.infrastructure.loaders.external.advanced_pdf_loader]



Reading PDF for file: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/nutrients-13-01241.pdf ...

2025-10-07T20:41:27.139496 [warning  ] Failed to process PDF with AdvancedPdfLoader: Unable to get page count. Is poppler installed and in PATH? [cognee.infrastructure.loaders.external.advanced_pdf_loader]

2025-10-07T20:41:27.140291 [info     ] Falling back to PyPDF loader for /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/nutrients-13-01241.pdf [cognee.infrastructure.loaders.external.advanced_pdf_loader]

2025-10-07T20:41:27.142512 [info     ] Reading PDF: /Users/daulet/Desktop/dev/cognee/examples/data/scientific_papers/nutrients-13-01241.pdf [cognee.infrastructure.loaders.external.pypdf_loader]

2025-10-07T20:41:27.289235 [info     ] Coroutine task completed: `ingest_data` [run_tasks_base]

2025-10-07T20:41:27.289621 [info     ] Coroutine task completed: `resolve_data_directories` [run_tasks_base]

2025-10-07T20:41:27.289895 [info     ] Pipeline run c


2025-10-07T20:41:27.349508 [info     ] Async Generator task started: `extract_chunks_from_documents` [run_tasks_base]

2025-10-07T20:41:27.387202 [info     ] Coroutine task started: `extract_graph_from_data` [run_tasks_base]

2025-10-07T20:41:27.398908 [info     ] Coroutine task started: `extract_graph_from_data` [run_tasks_base]

2025-10-07T20:41:48.152674 [info     ] No close match found for 'person' in category 'classes' [OntologyAdapter]

2025-10-07T20:41:48.153506 [info     ] No close match found for 'laura torres-collado' in category 'individuals' [OntologyAdapter]

2025-10-07T20:41:48.154051 [info     ] No close match found for 'laura maría compañ-gabucio' in category 'individuals' [OntologyAdapter]

2025-10-07T20:41:48.154641 [info     ] No close match found for 'sandra gonzález-palacios' in category 'individuals' [OntologyAdapter]

2025-10-07T20:41:48.155119 [info     ] No close match found for 'leyre notario-barandiaran' in category 'individuals' [OntologyAdapter]

2025-10-0

Q: What are common risk factors for Type 2 Diabetes?
A: ['Common risk factors for Type 2 Diabetes include obesity (as indicated by a body mass index of 25 kg/m² or more), sedentary lifestyle, poor dietary habits, smoking habits, and the presence of chronic conditions such as hypertension and high blood cholesterol. Additionally, waist circumference measurement can indicate risk levels, with increased risk seen in men with a waist over 102 cm and women over 88 cm.']

Q: What preventive measures reduce the risk of Hypertension?
A: ['Preventive measures to reduce the risk of hypertension include:\n1. **Moderate Coffee Consumption**: Regular moderate coffee consumption (1-4 cups per day) is associated with a decreased risk of developing hypertension.\n2. **Dietary Antioxidants**: Coffee contains antioxidants, such as chlorogenic acid, which may inhibit inflammation and support cardiovascular health.\n3. **Healthy Preparation Methods**: Choosing filtered coffee over boiled coffee can reduce

## Visualizing the Knowledge Graph

Let's visualize how our ontology connects different medical concepts:

In [6]:
import webbrowser
import os
from cognee.api.v1.visualize.visualize import visualize_graph
html = await visualize_graph()
home_dir = os.path.expanduser("~")
html_file = os.path.join(home_dir, "graph_visualization.html")
display(html_file)
webbrowser.open(f"file://{html_file}")


2025-10-07T20:42:22.959024 [info     ] Graph visualization saved as /Users/daulet/graph_visualization.html [cognee.shared.logging_utils]

2025-10-07T20:42:22.959720 [info     ] The HTML file has been stored on your home directory! Navigate there with cd ~ [cognee.shared.logging_utils]


'/Users/daulet/graph_visualization.html'

True

## Understanding the Results

The demonstration above shows how ontologies enhance our analysis by:

1. **Making Connections**: 
   - Linking related medical concepts even when not explicitly stated
   - Identifying relationships between symptoms, diseases, and risk factors

2. **Standardizing Terms**: 
   - Unifying different ways of referring to the same medical condition
   - Ensuring consistent terminology across documents

3. **Enabling Inference**: 
   - Drawing conclusions based on ontological relationships
   - Discovering implicit connections in the data

## Next Steps

To learn more about Cognee and ontologies:
1. Check out the [Cognee documentation](https://docs.cognee.ai/)
2. Explore more examples in the `examples` directory
3. Try creating your own domain-specific ontology

Remember to:
- Place your scientific papers in the appropriate directory
- Update the ontology path to point to your .owl file
- Replace the API key with your own OpenAI key

In [ ]:
# Only exit in interactive mode, not during GitHub Actions
import os

# Skip exit if we're running in GitHub Actions
if not os.environ.get('GITHUB_ACTIONS'):
    print("Exiting kernel to clean up resources...")
    os._exit(0)
else:
    print("Skipping kernel exit - running in GitHub Actions")

: 